In [1]:
import numpy as np
import sys
import time

# yeti modules
from preprocessing.igaparametrization import IGAparametrization, IGAmanip as manip
import postprocessing.postproc as pp
import reconstructionSOL as rsol

from preprocessing.igaparametrization import OPTmodelling

In [2]:
modeleIGA = IGAparametrization(filename='centrif_U1_cpl_U1_1load')


Reading Abaqus inp file from centrif_U1_cpl_U1_1load.inp...

Reading number of nodes by element...

Reading parameters defining the user elements...

Reading model name...
No model name.

Reading parameters...

Reading distributions...
0 distributions read.

Reading materials...
	Reading material Mat...
		Type isotropic.
		Elastic modulus = 210000.0
		Poisson = 0.3
	Material Mat successfully read.
	Reading material MatVoid...
		Type isotropic.
		Elastic modulus = 0.0
		Poisson = 0.0
	Material MatVoid successfully read.
2 materials read.

Reading parts...
	Reading part Piece...
		17 nodes read.
		1 elements read.
		2 elements read.
		3 elements read.
		5 groups read.
		3 properties read.
	Part Piece successfully read.
1 parts read.

Saving part objects...
Parts successfully saved.

Reading assembly...
Creating instance I1 of part Piece...
Successfully created.
0 nodes read.
1 instances created.

Reading initial boundary conditions...
	Initial step created.
Initial boundary conditions r

In [3]:
# Refinement to create optimisation model
nb_deg = np.zeros((3, modeleIGA._nb_patch),dtype=np.intp)
nb_ref = np.zeros((3, modeleIGA._nb_patch),dtype=np.intp)

nb_ref[:, 1] = np.array([0, 2, 0])
nb_deg[:, 1] = np.array([0, 1, 0])
additional_knots = {"patches": np.array([0]),
                    "1": np.array([]), "2": np.array([]), "3": np.array([])}


# Initial refinement (none)
modeleIGA.refine(nb_ref, nb_deg, additional_knots)

 Refinement has been successfully done.


In [4]:
modeleIGA._COORDS

array([[-0.75,  0.75, -0.75,  0.75, -0.75,  0.75, -0.75,  0.75, -0.75,
         0.75, -0.75,  0.75, -0.75,  0.75, -0.75,  0.75, -0.75,  0.75,
        -0.75,  0.75, -0.75,  0.75, -0.75,  0.75, -0.75,  0.75, -0.75,
         0.75, -0.75,  0.75, -0.75,  0.75,  0.  ],
       [ 0.  ,  0.  ,  1.2 ,  1.2 ,  0.  ,  0.  ,  1.2 ,  1.2 ,  1.2 ,
         1.2 ,  1.8 ,  1.8 ,  3.  ,  3.  ,  4.2 ,  4.2 ,  5.4 ,  5.4 ,
         6.  ,  6.  ,  1.2 ,  1.2 ,  1.8 ,  1.8 ,  3.  ,  3.  ,  4.2 ,
         4.2 ,  5.4 ,  5.4 ,  6.  ,  6.  ,  0.  ],
       [-0.75, -0.75, -0.75, -0.75,  0.75,  0.75,  0.75,  0.75, -0.75,
        -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75, -0.75,
        -0.75, -0.75,  0.75,  0.75,  0.75,  0.75,  0.75,  0.75,  0.75,
         0.75,  0.75,  0.75,  0.75,  0.75,  0.  ]])

In [5]:
icp = np.where(modeleIGA._COORDS[1, :] > 1.2)[0]
nb_var = np.unique(modeleIGA._COORDS[1, icp]).size
print(nb_var)

5


In [6]:
# Define shape change from design variables
# min and max dimensions, assuming that design variables are in [0,1]
mindim = 0.1
maxdim = 3.5

def shapemodif(coords0, igapara, var):
    igapara._COORDS[:,:] = coords0[:,:]
    # shape change is made on points with y coord higher than 3
    icp = np.where(modeleIGA._COORDS[1, :] > 1.2)[0]
    i = 0
    for y in np.unique(modeleIGA._COORDS[1, icp]):
        # WARNING exact real value comparison is unsafe
        jcp = np.where(modeleIGA._COORDS[1, :] == y)[0]
    
        #igapara._COORDS[0, jcp[0]] = - (mindim + var[i]*(maxdim-mindim))/2.
        igapara._COORDS[2, jcp[0]] = - (mindim + var[i]*(maxdim-mindim))/2.
        
        #igapara._COORDS[0, jcp[1]] =   (mindim + var[i]*(maxdim-mindim))/2.
        igapara._COORDS[2, jcp[1]] = - (mindim + var[i]*(maxdim-mindim))/2.
        
        #igapara._COORDS[0, jcp[2]] =   (mindim + var[i]*(maxdim-mindim))/2.
        igapara._COORDS[2, jcp[2]] =   (mindim + var[i]*(maxdim-mindim))/2.
        
        #igapara._COORDS[0, jcp[3]] = - (mindim + var[i]*(maxdim-mindim))/2.
        igapara._COORDS[2, jcp[3]] =   (mindim + var[i]*(maxdim-mindim))/2.
        
        i += 1
    return None

In [7]:
# Refinement from optim model to analysis model
nb_deg[:,0] = np.array([1, 1, 1])
nb_ref[:,0] = np.array([2, 2, 2])

nb_deg[:,1] = np.array([1, 0, 1])
nb_ref[:,1] = np.array([2, 2, 2])

nb_ref[:, 2] = np.array([2, 2, 0])
nb_deg[:, 2] = np.array([1, 1, 0])

In [8]:
# Define optim problem 
optPB = OPTmodelling(modeleIGA, nb_var, shapemodif,
                     nb_degreeElevationByDirection = nb_deg, 
                     nb_refinementByDirection      = nb_ref)

 Refinement has been successfully done.


In [9]:
# OPTIMISATION
from scipy.optimize import minimize

# Initial values of design variables
x0 = ((1.5-mindim)/(maxdim-mindim))*np.ones(nb_var)
# Compute initial values
v0 = optPB.compute_volume(x0)
c0 = optPB.compute_compliance_discrete(x0)

print('c0 : ', c0)
print('v0 : ', v0)

Compute Volume
Compute compliance
 Linear elasticity analysis...
--------------------
Patch  3 of type U5
  > coupling of face no.3 of patch  2
  > on face no.4 of patch  1
c0 :  2.1994409280355863e-05
v0 :  13.500000000000105


In [10]:
vol_var = 0.2

# Define functions and gradients
def volIGA(xk):
    global v0
    return (optPB.compute_volume(xk)-v0)/v0
def gradVolIGA(xk):
    global v0
    print("INFO gradVolIGA :")
    print("x : ", xk)
    print("grad : ", optPB.compute_gradVolume_AN(xk, listpatch=np.array([1,1,0])))
    return optPB.compute_gradVolume_AN(xk, listpatch=np.array([1,1,0]))/v0

def const_var_vol(x_k, *args):
    global v0
    v_k = optPB.compute_volume(x_k)
    return (v_k - (1 - vol_var) * v0) * ((1 + vol_var) * v0 - v_k)/(v0**2.)

def grad_const_var_vol(x_k, *args):
    global v0
    v_k = optPB.compute_volume(x_k)
    grad_v = optPB.compute_gradVolume_AN(x_k)
    return  2 * grad_v * (v0 - v_k)/(v0**2.)
    
def compIGA(xk):
    global c0
    return optPB.compute_compliance_discrete(xk)/c0

def gradCompIGA(xk):
    global c0
    print("INFO gradCompliance :")
    print("x : ", xk)
    print("grad : ", optPB.compute_gradCompliance_AN(xk))
    return optPB.compute_gradCompliance_AN(xk)/c0

In [11]:
iopt = 0

In [12]:
# Define callback function
def saveXk(xk):
    global iopt
    print(('\nIteration%i'%iopt))
    print(xk)
    print('C : ', optPB.compute_compliance_discrete(xk))
    print('V : ', optPB.compute_volume(xk))
    SOL,u = rsol.reconstruction(
            **optPB._fineParametrization.get_inputs4solution(optPB._save_sol_fine))
    pp.generatevtu(*optPB._fineParametrization.get_inputs4postprocVTU(
            'OPT3-fine%0.2d'%iopt,  SOL.transpose(), nb_ref=3*np.array([1,1,1]),
            Flag=np.array([True, False, False])))
    
    iopt += 1
    
    return None

In [13]:
# Bounds for design variables
bds = ((0., 1.),)*nb_var
constraint = ({'type': 'eq', 'fun':volIGA, 'jac':gradVolIGA})
#constraint = ({'type': 'eq', 'fun':volIGA})
#constraint = [{'type': 'ineq', 'fun': const_var_vol, 'jac': grad_const_var_vol}]
x0 = ((1.5-mindim)/(maxdim-mindim))*np.ones(nb_var)
print('C : ', optPB.compute_compliance_discrete(x0))
print('V : ', optPB.compute_volume(x0))

res = minimize(compIGA, x0, method='SLSQP',
               jac=gradCompIGA, bounds=bds,
               constraints=constraint, callback=saveXk)

#res = minimize(compIGA, x0, method='SLSQP',
#               jac=gradCompIGA, bounds=bds,
#               constraints=constraint, callback=saveXk)

print(res)
print('C : ', optPB.compute_compliance_discrete(res['x']))
print('V : ', optPB.compute_volume(res['x']))

Compute compliance
C :  2.1994409280355863e-05
Compute Volume
V :  13.500000000000105
Compute Volume
Compute compliance
INFO gradCompliance :
x :  [0.41176471 0.41176471 0.41176471 0.41176471 0.41176471]
Compute AN gradient of the compliance
grad :  [-9.02451980e-06 -8.73758219e-06 -3.67292813e-06 -7.79083075e-07
 -9.27282415e-08]
Compute AN gradient of the compliance
Compute Volume
INFO gradVolIGA :
x :  [0.41176471 0.41176471 0.41176471 0.41176471 0.41176471]
Compute AN gradient of the volume
grad :  [4.08 6.12 6.12 4.08 2.04]
Compute AN gradient of the volume
Compute compliance
 Linear elasticity analysis...
--------------------
Patch  3 of type U5
  > coupling of face no.3 of patch  2
  > on face no.4 of patch  1
Compute Volume

Iteration0
[0.63033726 0.52142275 0.29115268 0.25544942 0.32011212]
Compute compliance
C :  1.7792219661837688e-05
Compute Volume
V :  13.500000000000059
 Post processing ...
INFO gradCompliance :
x :  [0.63033726 0.52142275 0.29115268 0.25544942 0.32011212

  --> File OPT3-fine08.vtu has been created.
V :  13.50000000000022
